In [5]:
import Snapshot_code
import pandas as pd

new_df2 = pd.read_csv('Data/final_file.csv')

mapping_dict = {'Baseline':'US_Population','Segment_1':'Application Completed'}

# Set up variables for snapshot
file_name = 'Unlock_11182022_Application Completed_1108'
seg_var = 'category'
epsilon_path = 'Data/Epsilon_Final.xlsx'
bin_vars_path = 'Data/Epsilon_attributes_binning_2.csv'
# Read in file and set bins


profile = Snapshot_code.Snapshot_Profile(profile_data = new_df2, segment_var=seg_var, continuous_path = bin_vars_path,segments = None, segment_names = None, include = None,variable_order = None, other_segment = False,
                 file = file_name, exclude = [], PPT = True, continuous = [], excludeother = False,mapping_dict=mapping_dict)

20221025
<Worksheet "DS_MAGAZINE_LIFE_QUINTILE"> removed
<Worksheet "DS_GREEN_THUMB_QUINTILE"> removed
<Worksheet "DS_GIFTS_FOR_ALL_QUINTILE"> removed
<Worksheet "VEHICLE_YEAR_4"> removed
<Worksheet "VEHICLE_YEAR_3"> removed
<Worksheet "VEHICLE_STYLE_CODE_4"> removed
<Worksheet "VEHICLE_MILEAGE_CODE_3"> removed
<Worksheet "VEHICLE_MANUFACTURING_CODE_4"> removed
<Worksheet "VEHICLE_MAKE_4"> removed
<Worksheet "VEHICLE_MAKE_3"> removed
<Worksheet "VEHICLE_LAST_SEEN_DATE_4"> removed
<Worksheet "VEHICLE_LAST_SEEN_DATE_3"> removed
<Worksheet "VEHICLE_FIRST_SEEN_DATE_4"> removed
<Worksheet "VEHICLE_FIRST_SEEN_DATE_3"> removed
<Worksheet "VEHICLE_CLASS_CODE_4"> removed
<Worksheet "VEHICLE_CLASS_CODE_3"> removed
<Worksheet "VEHICLE_YEAR"> removed
<Worksheet "BIRTHDATE_OF_2ND_CHILD_ENHANCED"> removed
<Worksheet "PERSON_KEY_3"> removed
<Worksheet "STREET_POST_DIRECTION"> removed
<Worksheet "TOOLS_ELEC_QUINT"> removed
<Worksheet "MID_TKT_F_APPRL_ACC_QUINT"> removed
<Worksheet "NEW_MARRIED_CHANGE_